In [236]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

regfull = pd.read_excel("201720&201730 Fall Makeup List (2018-07-24) - DrHamad.xlsx",sheetname="StudentList")

regfull.columns

Index(['Term', 'Campus', 'Student Id', 'Full Name', 'Final Approval ',
       'Division', 'Course-Code', 'Equivalent Course?', 'Course Title',
       'Final Course', 'FWA STATUS', 'Exam Delivery', 'Duration (hr)',
       'Duration (min)'],
      dtype='object')

In [237]:
reg = regfull

In [238]:
reg["Student Id"]=reg["Student Id"].astype(str) 
reg['ExamSlot'] = 0
reg["Day"] = 0
reg['nStudent'] = 0


courses = reg["Final Course"].unique()


#This loop calculated number of students registered in a course
for course in courses:
    reg['nStudent'][reg["Final Course"]==course] = len(reg[reg["Final Course"]==course])

# This function schedule a gevin course using a day and slot

def scheduleCourse(day,slot):
    courses = reg[(reg["Day"] == 0)]["Final Course"].unique() # focusing on not scheduled courses
    
    for course in courses:
        
        # The following 5 lines are appending the students registered in a given course 
        # to those who are alreday scheduled on the day and finding how many students are
        # scheduled 3 exams.  If any then this day and slot are not good.
        # This code limit maximum of exams a student can do in a day to 2.
        # you can change the number 3 to any number based on your college policy
        
        reg3 = reg[(reg["Final Course"]==course)]  
        nreg = reg[(reg["Day"] == day)]
        rg = reg3.append(nreg)  
        rgg = rg.groupby('Student Id').count()
        x = len(rgg[rgg['Campus']==3])
        
        dayslot = len(reg[(reg["Day"]==day) & (reg["ExamSlot"]==slot)]["Final Course"].unique())
        
        if(dayslot<40):  #This is the maximum of exams in one slot
        
            if(x==0):
                # The same process above is repeated for the exam slot
                sreg3 = reg[(reg["Final Course"]==course)]
                snreg = reg[(reg["ExamSlot"] == slot) & (reg["Day"]==day)]
                srg = sreg3.append(snreg)
                srgg = srg.groupby('Student Id').count()
                sx = len(srgg[srgg['Campus']==2])         
                if(sx==0):
                    reg["ExamSlot"][reg["Final Course"]==course]=slot  # scheduling the course on the given slot
                    reg["Day"][reg["Final Course"]==course]=day  #scheduling the course on the given day

In [239]:
# The main code to build the schedule

reg = reg.sort_values(['Division','Final Course','nStudent'],ascending=[1,0,0]) #sorting by division, course, nStudent

reg['ExamSlot'] = 0  #reseting the ExamSlot
reg["Day"] = 0       #reseting the Day

# The following lines go day by day and slot by slot looking for a space that does
# not create conflict for any student

for day in range(1,9): #8 days
    for slot in range(1,4): #3 exams slots
        scheduleCourse(day,slot)
        print("Day ", day, "Slot = ", slot, len(reg[(reg["Day"] == day) & (reg["ExamSlot"] == slot)]["Final Course"].unique()))

print(reg[reg["Day"]==0]["Final Course"].unique()) #Display any course that was not schedulled.


#Formating the schedule.

schedule = pd.DataFrame(courses, columns=["Course"])
schedule["ExamDay"] = 0
schedule["ExamSlot"] = 0
schedule["Duration (min)"] = 0
schedule["Exam Delivery"] = ""

for course in courses:
    xday = reg[reg["Final Course"]==course]["Day"].unique()
    xslot = reg[reg["Final Course"]==course]["ExamSlot"].unique()
    xdur = reg[reg["Final Course"]==course]["Duration (min)"].unique()
    xdel = reg[reg["Final Course"]==course]["Exam Delivery"].unique()
    
    schedule["ExamDay"][schedule["Course"]==course] = xday[0]
    schedule["ExamSlot"][schedule["Course"]==course] = xslot[0]
    schedule["Duration (min)"][schedule["Course"]==course] = xdur[0]
    schedule["Exam Delivery"][schedule["Course"]==course] = xdel[0]
    
    
schedule.sort_values(['ExamDay','ExamSlot'], inplace=True)   

writer = pd.ExcelWriter('ExamSchedule.xlsx')
reg.to_excel(writer,'Deatils')
schedule.to_excel(writer,'Schedule')
writer.save()

Day  1 Slot =  1 40
Day  1 Slot =  2 40
Day  1 Slot =  3 40
Day  2 Slot =  1 40
Day  2 Slot =  2 35
Day  2 Slot =  3 9
Day  3 Slot =  1 5
Day  3 Slot =  2 1
Day  3 Slot =  3 0
Day  4 Slot =  1 1
Day  4 Slot =  2 0
Day  4 Slot =  3 0
Day  5 Slot =  1 0
Day  5 Slot =  2 0
Day  5 Slot =  3 0
Day  6 Slot =  1 0
Day  6 Slot =  2 0
Day  6 Slot =  3 0
Day  7 Slot =  1 0
Day  7 Slot =  2 0
Day  7 Slot =  3 0
Day  8 Slot =  1 0
Day  8 Slot =  2 0
Day  8 Slot =  3 0
[]
